## Simulation of a M/M/1 queue using processes

This simulation is adapted from the Bank Renege example in the documentation of a previous version of SimJulia: http://simjuliajl.readthedocs.io/en/stable/examples/1_bank_renege.html

In [ ]:
using SimJulia
using Distributions
using ResumableFunctions
using RDST, Random

Let's first simulate a fixed number of clients.

In [ ]:
const RANDOM_SEED = 200
const NEW_CUSTOMERS = 5  # Total number of customers
const INTERVAL_CUSTOMERS = 2.0  # Generate new customers roughly every x seconds
const MEAN_SERVICE = 1.9

# The macro @resumable allows to suspend a function until some event wakes it up.
@resumable function source(env::Simulation, number::Int, interval::Float64, counter::Resource)
    d = Exponential(interval)
    for i in 1:number
        # The customer service time is random. During the service, the counter
        # is not available to any other customer.
        @yield timeout(env, rand(d))
        @process customer(env, i, counter, MEAN_SERVICE)
    end
end

@resumable function customer(env::Simulation, idx::Int, counter::Resource, time_in_system::Float64)
    # Record the arrival time in the system
    arrive = now(env)
    println("$arrive: arrival of customer $idx")
    @yield request(counter)
    # The simulation clock now contains the time when the client goes to the server.
    wait = now(env) - arrive
    # Record the waiting time
    waits[idx] = wait
    println("$(now(env)): customer $idx has waited $wait")
    @yield timeout(env, rand(Exponential(time_in_system)))
    println("$(now(env)): customer $idx: finished")
    @yield release(counter)
end


In [ ]:
# Setup and start the simulation
println("M/M/1 with processes")
waits = zeros(NEW_CUSTOMERS)

In [ ]:
Random.seed!(RANDOM_SEED)
env = Simulation()

In [ ]:
# Start processes and run
counter = Resource(env, 1)
@process source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter)

In [ ]:
run(env)

We can compute the mean waiting time by

In [ ]:
mean(waits)

However, most of the time, we do not know the number of client. We first set the end of simulation event by specifying an time horizon when running the simulation.

In [ ]:
@process source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter)
run(env, 5.0)

We observe however that the simulation time has not been reset to 0. A simple solution is to create a new simulation environment. This also requires to set the resource again.

In [ ]:
env = Simulation()
counter = Resource(env, 1)
@process source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter)
run(env, 5.0)

The random draws are different but we can produce the same as previously by using the same seed, i.e. the same initial state.

In [ ]:
Random.seed!(RANDOM_SEED)
env = Simulation()
counter = Resource(env, 1)
@process source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter)
run(env, 15.0)

However, a possible issue is that a customer never finishes his service. If we want to ensure that the customer complete his journey in the system, we have to modify the source function. We can circumvent it by redefining the source function so that no customer is generated after a horizon limit, but we do not put a limit when calling the run function.

In [ ]:
@resumable function source!(env::Simulation, number::Int, interval::Float64, counter::Resource, limit::Float64, nserved::Array{Int64,1})
    nserved[1] = 0
    d = Exponential(interval)
    for i in 1:number
        @yield timeout(env, rand(d))
        if (now(env) > limit) break end
        @process customer(env, i, counter, MEAN_SERVICE)
        nserved[1] += 1
    end
end

In [ ]:
nserved = [ 0 ]
Random.seed!(RANDOM_SEED)
env = Simulation()
counter = Resource(env, 1)

@process source!(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter, 5.0, nserved)
run(env)

This raises the question: should we flush the entities in the system at the end of the horizon of allow the entities in the system to complete their process? It depends on the context!

In our case, the mean waiting time is

In [ ]:
mean(waits[1:nserved[1]])

In [ ]:
@resumable function new_source!(env::Simulation, interval::Float64, counter::Resource, limit::Float64, nserved::Array{Int64,1})
    nserved[1] = 0
    i = 0
    d = Exponential(interval)
    while (true)
        @yield timeout(env, rand(d))
        if (now(env) > limit) break end
        i += 1
        @process new_customer(env, i, counter, MEAN_SERVICE, new_waits)
    end
    nserved[1] = i
 end

In [ ]:
@resumable function new_customer(env::Simulation, idx::Int, counter::Resource, time_in_system::Float64, waits::Array{Float64,1})
    # Record the arrival time in the system
    arrive = now(env)
    println("$arrive: arrival of customer $idx")
    @yield request(counter)
    # The simulation clock now contains the time when the client goes to the server.
    wait = now(env) - arrive
    # Record the waiting time
    waits = push!(waits, wait)
    println("$(now(env)): customer $idx has waited $wait")
    @yield timeout(env, rand(Exponential(time_in_system)))
    println("$(now(env)): customer $idx: finished")
    @yield release(counter)
end

In [ ]:
nserved = [ 0 ]
Random.seed!(RANDOM_SEED)
env = Simulation()
counter = Resource(env, 1)
new_waits = Float64[]

@process new_source!(env, INTERVAL_CUSTOMERS, counter, 5.0, nserved)

run(env)

In [ ]:
new_waits

In [ ]:
nserved[1]